<a href="https://colab.research.google.com/github/Strojove-uceni/23206-final-emergent/blob/main/EMERGENT.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# EMERGENT: barcodE iMage EnhanceR usinG dEep Neural neTworks

Autoři: Alexej Gaj, Štěpán Studenovský

---
Cílem tohoto projektu bylo vytvořit program na rozpoznávání a čtení čárových EAN a QR kódů, který by byl integrovatelný do zařízení používající fotoaparát. Volili jsme dva přístupy řešení, první cesta byla jít přes:

> detekce kódu -> vyříznutí -> denoising -> čtení

> denoising -> detekce -> vyříznutí -> čtení

K detekci kódu jsme se snažili použít nejprve vlastní síť vytvořenou from scratch se špatnými výsledky, tak jsme zvolili finetunování modelu Yolov5s/m. Vyříznutí jsme napsali simple python skript, který vezme výsledky z yolov5 a vyřízne obrázek.

... detekce - todo

Čtení kódu jsme použili již implementovanou čtečku kódu viz. odkaz???

#Dataset

Pro detekci čárových kódu jsme nejprve připravili datasety, některé již anotované: https://www.kaggle.com/datasets/whoosis/barcode-detection-annotated-dataset, http://artelab.dista.uninsubria.it/downloads/datasets/barcode/medium_barcode_1d/medium_barcode_1d.html,

použili jsme ještě jeden dataset ze stránky kaggle.com, kde bylo přibližně 500 obrázků anotovaných  tedď již ručně pomocí https://www.makesense.ai/, výborný nastroj pro anotování obrázků pro formát YOLO.

Taktéž jsme se věnovali uměleému generování barcode pomocí ....

Zašumění obrázků....


# Metodologie

Co se detekce čárových kódů a typu úlohy, bylo by nejjednodušší navrhnout nebo použít sítě typu Yolo, fast RNN.... Síť by měla detekovat čárový kód, udělat kolem něj obdélník, pojmenovat třídu, do které jej klasifikovala a poslat výsledek predikce. Taktéž by zaznamenala kde se výřez na fotce vyskytuje pro budoucí vystřihování.

Jelikož se jedná o poměrně jednoduchou úlohu detekce jen jedné třídy, tak jsme nejprve zvolili cestu pro vytvoření a natrénování vlastní neuronové sítě v jazyce pytorch-lightning. Trénovali jsme přes Helios na FJFI, dále v colab.google na T4. Vlastní síť se neukázala příliš efektivní, nejspíše by to chtělo mnohem složitější architekturu sítě, vytvořit vlastní loss funkci a hlubší porozumění problému - https://colab.research.google.com/drive/1dq53A7OU2OXf5SzzKatDl7_9VUder5bu?usp=sharing.py.

Výsledky trénování a testování jsme ukládali pomocí Weights and Biases.

Druhý pokus byl o dotrénování modelu Yolov5s, pokus byl opět neúspěšný. Snaha byla pouze načíst model a vytvořit vlastní přístup a prostředí pro dotrénování - viz AlexNet na cvikách. Nepodařilo se díky nezkušenosti, neschopnosti pracujícího a hlavně díky komplikovanosti modelu implementovat jednotlivé prvky - loss funkci. Odkaz na colab - https://colab.research.google.com/drive/1GKQUOvoa0GSPtYpWARyyaOSIZxiRiaVB?usp=sharing

Poslední pokus jsme se věnovali postupu doporučenému od Ultralytics na stránce https://docs.ultralytics.com/yolov5/tutorials/train_custom_data/#next-steps. Zde jsme již po další úpravě datových souborů a následování tutoriálu došli k výsledku v https://colab.research.google.com/drive/1Kn7DAfYJPsnvY6c9FK12aPYXOgdDtO0O?usp=sharing. Výsledky trénování se již ukázali jako úspěšné a barcode klasifikoval správně.


Vyříznutí obrázku, jsme implemetovali jednoduchý Python skript, který si bere predikovaný .txt/.json výstup z Yolov5 a ořízne obrázek dle klasifikace. https://colab.research.google.com/drive/1tx5v-YymIFhwkDBcqea0apRgVOBslwgE?usp=sharing


In [1]:
#ps: chybí uložení modelu, nevím jak se to dělá, vhodit do toho nějaký obrázek a vytahnout ty výstupy, který se pak musí nějak nahrát do toho Image_crop...

Asi by to chtělo nějaký demo, ale vzhledem k tomu že nic nefunguje tak nevím xd pokusím se to rozchodit, ale ta vzájemná integrace bude asi pain, to upřímně mco neumím. Asi bude lepší když to uděláme jen formou obrázků so to vrací a na nějakou integraci se vysereme xdd